In [ ]:
import numpy as np
from datetime import datetime
import smtplib
import time
from selenium import webdriver
#For Prediction
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, cross_validation, svm
#For Stock Data
from iexfinance import Stock
from iexfinance import get_historical_data

#Navigating to the Yahoo stock screener
driver = webdriver.Chrome(executable_path='/Users/aniketmuluk/Documents/chromedriver')


url = 'https://finance.yahoo.com/screener/predefined/aggressive_small_caps?offset=0&count=202'

driver.get(url)

stock_list = []
for i in range(1, 100):
    ticker = driver.find_element_by_xpath('//*[@id ="scr-res-table"]/div[2]/table/tbody/tr['+str(i)+']/td[1]/a')
    stock_list.append(ticker.text)

print len(stock_list)
print(stock_list)

for i in stock_list:
    try:
        predictData(i, 5)
    except:
        print("Stock: " + i + " was not predicted")

start = datetime(2017, 1, 1)
end = datetime.now()
#Outputting the Historical data into a .csv for later use
df = get_historical_data("BZUN", start=start, end=end,     output_format='pandas')
df

csv_name = ('Baozun_Inc.csv')
df.to_csv(csv_name)
df['prediction'] = df['close'].shift(-1)
df.dropna(inplace=True)
#forecast_time = int(days)

X = np.array(df.drop(['prediction'], 1))
Y = np.array(df['prediction'])
X = preprocessing.scale(X)
X_prediction = X[-5:]
X_train, X_test, Y_train, Y_test =         cross_validation.train_test_split(X, Y, test_size=0.5)

#Performing the Regression on the training data
clf = LinearRegression()
clf.fit(X_train, Y_train)
prediction = (clf.predict(X_prediction))

df